In [17]:
import numpy as np


class Tensor(object):

    
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None

        
    def backward(self, grad):
        self.grad = grad
        
        if (self.creation_op == 'add'):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
    
        
    def __add__(self, other):
        return Tensor(self.data + other.data, creators=[self, other], creation_op='add')
    
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    
    def __str__(self):
        return str(self.data.__str__())

In [18]:
x = Tensor([1, 2, 3, 4, 5])
y = Tensor([2, 2, 2, 2, 2])

z = x + y
print(z)

[3 4 5 6 7]


In [21]:
z.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


ошибка

In [22]:
a = Tensor([1, 2, 3, 4, 5])
b = Tensor([2, 2, 2, 2, 2])
c = Tensor([5, 4, 3, 2, 1])
d = a + b
e = b + c
f = d + e
f.backward(Tensor([1, 1, 1, 1, 1]))
print(b.grad)

[1 1 1 1 1]


# SECTION 2: autograd

In [134]:
class Tensor(object):

    
    def __init__(self, data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        
        if id is None:
            id = np.random.randint(0, 100000)
        self.id = id
        
        if creators is not None:
            for c in creators:
                if self.id not in c.children:
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
                    
                    
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if cnt != 0:
                return False
        return True
        
        
    def backward(self, grad=None, grad_origin=None):
        if self.autograd:
            if grad is None:
                grad = Tensor(np.ones_like(self.data))
            
            if grad_origin is not None:
                if self.children[grad_origin.id] == 0:
                    raise Exception('cannot backprop more than once')
                else:
                    self.children[grad_origin.id] -= 1
            
            if self.grad is None:
                self.grad = grad
            else:
                self.grad += grad
            
            if self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None):
                if self.creation_op == 'add':
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                
                if self.creation_op == 'neg':
                    self.creators[0].backward(self.grad.__neg__())
                    
                if self.creation_op == 'sub':
                    new = Tensor(self.grad.data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.__neg__().data)
                    self.creators[1].backward(new, self)
                
                if self.creation_op == 'mul':
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new, self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)
                    
                if self.creation_op == 'mm':
                    act = self.creators[0]     # usually an activation
                    weights = self.creators[1] # usually a weight matrix
                    new = self.grad.mm(weights.transpose())
                    act.backward(new)
                    new = self.grad.transpose().mm(act).transpose()
                    weights.backward(new)

                if self.creation_op == 'transpose':
                    self.creators[0].backward(self.grad.transpose())
                    
                if 'sum' in self.creation_op:
                    dim = int(self.creation_op.split('_')[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim,ds))
                    
                if 'expand' in self.creation_op:
                    dim = int(self.creation_op.split('_')[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if self.creation_op == 'sigmoid':
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                    
                if self.creation_op == 'tanh':
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
    
                if self.creation_op == 'index_select':
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))

                if self.creation_op == 'cross_entropy':
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))


    def __add__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self, other],
                          creation_op='add')
        return Tensor(self.data + other.data)

    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op='neg')
        return Tensor(self.data * -1)
    
   
    def __sub__(self, other):
        if self.autograd and other.autograd: 
            return Tensor(self.data - other.data, 
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)

    
    def __mul__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)

    
    def sum(self, dim):
        if self.autograd:
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))

    
    def expand(self, dim, copies):
        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)
        if self.autograd:
            return Tensor(new_data, autograd=True, creators=[self], creation_op="expand_"+str(dim))
        return Tensor(new_data)

    
    def transpose(self):
        if self.autograd:
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        return Tensor(self.data.transpose())

    
    def mm(self, x):
        if self.autograd:
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    
    def sigmoid(self):
        if self.autograd:
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op='sigmoid')
        return Tensor(1 / (1 + np.exp(-self.data)))
    
    
    def tanh(self):
        if self.autograd:
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op='tanh')
        return Tensor(np.tanh(self.data))

    
    def index_select(self, indices):
        if self.autograd:
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op='index_select')
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
                                              
                                              
    def cross_entropy(self, target_indices):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
    
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
        
        if self.autograd:
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op='cross_entropy')
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out
        return Tensor(loss)                                              
    
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    
    def __str__(self):
        return str(self.data.__str__())

In [116]:
a = Tensor([1, 2, 3, 4, 5], autograd=True)
b = Tensor([2, 2, 2, 2, 2], autograd=True)
c = Tensor([5, 4, 3, 2, 1], autograd=True)
d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1, 1, 1, 1, 1])))
print(b.grad)

[2 2 2 2 2]


In [84]:
print(a.id, b.id, c.id, d.id, e.id, f.id)

23630 76945 64305 32347 96020 52018


In [85]:
b.children

{32347: 0, 96020: 0}

# SECTION 3: manual network

In [91]:
np.random.seed(0)

data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
target = np.array([[0], [1], [0], [1]])

weights_0_1 = np.random.rand(2, 3)
weights_1_2 = np.random.rand(3, 1)

for i in range(10):
    layer_1 = data.dot(weights_0_1)
    layer_2 = layer_1.dot(weights_1_2)

    diff = layer_2 - target
    sqdiff = diff * diff
    loss = sqdiff.sum(0)
    
    layer_1_grad = diff.dot(weights_1_2.transpose())
    weight_1_2_update = layer_1.transpose().dot(diff)
    weight_0_1_update = data.transpose().dot(layer_1_grad)
    
    weights_1_2 -= weight_1_2_update * 0.1
    weights_0_1 -= weight_0_1_update * 0.1
    
    print(loss[0])

5.066439994622395
0.4959907791902342
0.4180671892167177
0.35298133007809646
0.2972549636567377
0.24923260381633286
0.20785392075862477
0.17231260916265181
0.14193744536652994
0.11613979792168386


# SECTION 4: framework

In [92]:
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)

w = list()
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))

for i in range(10):
    # Predict
    pred = data.mm(w[0]).mm(w[1])
    # Compare
    loss = ((pred - target)*(pred - target)).sum(0)
    # Learn
    loss.backward(Tensor(np.ones_like(loss.data)))
    for w_ in w:
        w_.data -= w_.grad.data * 0.1
        w_.grad.data *= 0
    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]


# SECTION 5: SGD

In [93]:
class SGD(object):

    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
        
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
            
    
    def step(self, zero=True):
        for p in self.parameters:
            p.data -= p.grad.data * self.alpha
            
            if zero:
                p.grad.data *= 0

In [94]:
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)

w = list()
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))

optim = SGD(parameters=w)
for i in range(10):
    # Predict
    pred = data.mm(w[0]).mm(w[1])
    # Compare
    loss = ((pred - target)*(pred - target)).sum(0)
    # Learn
    loss.backward(Tensor(np.ones_like(loss.data)))
    
    optim.step()
    
    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]


# SECTION 6: Layers

In [95]:
class Layer(object):
    
    
    def __init__(self):
        self.parameters = list()
        
    
    def get_parameters(self):
        return self.parameters

In [99]:
class Linear(Layer):

    
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/n_inputs)
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        self.parameters.append(self.bias)
        
        
    def forward(self, input):
        return input.mm(self.weight) + self.bias.expand(0, len(input.data))

In [100]:
class Sequential(Layer):
    
    
    def __init__(self, layers=list()):
        super().__init__()
        self.layers = layers
        
        
    def add(self, layer):
        self.layers.append(layer)
        
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params

In [101]:
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)
model = Sequential([Linear(2,3), Linear(3, 1)])
optim = SGD(parameters=model.get_parameters(), alpha=0.05)
for i in range(10):
    pred = model.forward(data)
    loss = ((pred - target)*(pred - target)).sum(0)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    
    print(loss)

[2.33428272]
[0.06743796]
[0.0521849]
[0.04079507]
[0.03184365]
[0.02479336]
[0.01925443]
[0.01491699]
[0.01153118]
[0.00889602]


# SECTION 7: error layers

In [102]:
class MSELoss(Layer):
    
    
    def __init(self):
        super().__init__()
        
    
    def forward(self, pred, target):
        return ((pred - target)*(pred - target)).sum(0)

In [103]:
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)
model = Sequential([Linear(2,3), Linear(3, 1)])
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.05)
for i in range(10):
    pred = model.forward(data)
    loss = criterion.forward(pred, target)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    
    print(loss)

[2.33428272]
[0.06743796]
[0.0521849]
[0.04079507]
[0.03184365]
[0.02479336]
[0.01925443]
[0.01491699]
[0.01153118]
[0.00889602]


# SECTION 8: activate func

In [142]:
class Tanh(Layer):
    
    
    def __init__(self):
        super().__init__()
        
        
    def forward(self, input):
        return input.tanh()

In [111]:
class Sigmoid(Layer):
    
    
    def __init__(self):
        super().__init__()
        
    
    def forward(self, input):
        return input.sigmoid()

In [143]:
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)
model = Sequential([Linear(2,3), Tanh(), Linear(3, 1), Sigmoid()])
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=1)
for i in range(10):
    pred = model.forward(data)
    loss = criterion.forward(pred, target)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    
    print(loss)

[1.06372865]
[0.75148144]
[0.57384259]
[0.39574294]
[0.2482279]
[0.15515294]
[0.10423398]
[0.07571169]
[0.05837623]
[0.04700013]


# SECTION 9: Vector layer

In [137]:
class Embedding(Layer):
    
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        weight = (np.random.rand(vocab_size, dim) - 0.5) / dim
        self.weight = Tensor(weight, autograd=True)
        
        self.parameters.append(self.weight)
        
    
    def forward(self, input):
        return self.weight.index_select(input)

In [138]:
data = Tensor(np.array([1, 2, 3, 4]), autograd=True)
target = Tensor(np.array([[0], [1], [0], [1]]), autograd=True)
embed = Embedding(5, 3)
model = Sequential([embed, Tahn(), Linear(3, 1), Sigmoid()])
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.5)
for i in range(10):
    pred = model.forward(data)
    loss = criterion.forward(pred, target)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    
    print(loss)

[0.940077]
[0.82715916]
[0.71236228]
[0.59177034]
[0.47262476]
[0.36654053]
[0.28122701]
[0.21729468]
[0.17092777]
[0.13743206]


# SECTION 10: cross entropy

In [139]:
class CrossEntropyLoss(object):
    
    
    def __init__(self):
        super().__init__()
    
    
    def forward(self, input, target):
        return input.cross_entropy(target)

In [141]:
np.random.seed(0)

data = Tensor(np.array([1, 2, 1, 2]), autograd=True)
target = Tensor(np.array([0, 1, 0, 1]), autograd=True)
model = Sequential([Embedding(3, 3), Tahn(), Linear(3, 4)])
criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.1)
for i in range(10):
    pred = model.forward(data)
    loss = criterion.forward(pred, target)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    
    print(loss)

1.3885032434928422
0.9558181509266037
0.6823083585795604
0.509525996749312
0.39574491472895856
0.31752527285348275
0.2617222861964217
0.2206128392395422
0.18946427334830074
0.16527389263866668


# SECTION 11: RNN

In [145]:
class RNNCell(Layer):
    
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()
        
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if activation == 'sigmoid':
            self.activation = Sigmoid()
        elif activation == 'tanh':
            self.activation = Tanh()
        else:
            raise Exception('unknown non-linear function')
        
        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()
        
        
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size, self.n_hidden)), autograd=True)

In [155]:
import sys,random,math
from collections import Counter


def words2indices(sentence):
    return [word2index[word] for word in sentence]


with open('../part12/tasksv11/en/qa1_single-supporting-fact_train.txt','r') as f:
    raw = f.readlines()

tokens = [line.lower().replace("\n","").split(" ")[1:] for line in raw[0:1000]]
new_tokens = [(['-'] * (6 - len(line)) + line) for line in tokens]
tokens = new_tokens

vocab = list({word for sent in tokens for word in sent})
word2index = {word:i for i,word in enumerate(vocab)}

indices = [[word2index[w] for w in line] for line in tokens]
data = np.array(indices)

In [157]:
data

array([[42, 12, 57, 15, 53,  2],
       [42, 80, 61, 15, 53, 58],
       [42, 42, 38, 34, 76, 65],
       ...,
       [42, 26, 61, 15, 53, 47],
       [42, 42, 38, 34, 76, 51],
       [42, 26, 61, 15, 53, 46]])

In [172]:
embed = Embedding(vocab_size=len(vocab), dim=16)
model = RNNCell(n_inputs=16, n_hidden=16, n_output=len(vocab))
criterion = CrossEntropyLoss()
params = model.get_parameters() + embed.get_parameters()
optim = SGD(parameters=params, alpha=0.05)

BATCH_SIZE = 100
ITERATIONS = 1000

for iter in range(ITERATIONS):
    total_loss = 0
    hidden = model.init_hidden(batch_size=BATCH_SIZE)
    for t in range(5):
        input = Tensor(data[0:BATCH_SIZE, t], autograd=True)
        rnn_input = embed.forward(input=input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
    
    target = Tensor(data[0:BATCH_SIZE, t+1], autograd=True)
    loss = criterion.forward(output, target)
    loss.backward()
    optim.step()
    total_loss += loss.data
    if iter % 200 == 0:
        p_correct = (target.data == np.argmax(output.data, axis=1)).mean()
        print_loss = total_loss / (len(data) / BATCH_SIZE)
        print("Loss: {}, Correct: {}".format(print_loss, p_correct))

Loss: 0.48676869696249386, Correct: 0.03
Loss: 0.17432683969742818, Correct: 0.28
Loss: 0.15481171248806253, Correct: 0.32
Loss: 0.14192064292570747, Correct: 0.36
Loss: 0.13772838333569867, Correct: 0.36


In [173]:
BATCH_SIZE = 1
hidden = model.init_hidden(batch_size=BATCH_SIZE)
for t in range(5):
    input = Tensor(data[0:BATCH_SIZE, t], autograd=True)
    rnn_input = embed.forward(input=input)
    output, hidden = model.forward(input=rnn_input, hidden=hidden)
target = Tensor(data[0:BATCH_SIZE, t+1], autograd=True)
loss = criterion.forward(output, target)
ctx = ''
for idx in data[0:BATCH_SIZE][0][0:-1]:
    ctx += vocab[idx] + ' '
print("Context:", ctx)
print("Pred:", vocab[output.data.argmax()])

Context: - mary moved to the 
Pred: office.
